# Supervised Machine Learning Model to Predict CO2 Level

In [32]:
import numpy as np
import pandas as pd
import regex as re
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix, classification_report

In [33]:
# Load data
file_path = "./Data file/usa_co2_temp_1800_2013.csv"
co2_temp_df = pd.read_csv(file_path)
co2_temp_df

,country,year,co2,co2_per_capita,cement_co2,cement_co2_per_capita,coal_co2,coal_co2_per_capita,flaring_co2,gas_co2,...,co2_per_gdp,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,energy_per_gdp,Avg Temperature,Avg Temperature Uncertainty
0,United States,1800,0.253,0.042,NaN,NaN,0.253,0.042,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6000000,NaN,NaN,12.377868,3.205628
1,United States,1801,0.267,0.044,NaN,NaN,0.267,0.044,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6113782,NaN,NaN,12.681863,2.895417
2,United States,1802,0.289,0.046,NaN,NaN,0.289,0.046,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6229723,NaN,NaN,12.859703,3.065703
3,United States,1803,0.297,0.047,NaN,NaN,0.297,0.047,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6347862,NaN,NaN,12.692603,3.065032
4,United States,1804,0.333,0.052,NaN,NaN,0.333,0.052,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6468241,NaN,NaN,12.576432,3.379515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,United States,2009,5478.210,17.885,29.615,0.097,1905.943,6.222,39.071,1245.742,...,0.368,649.89,2.122,241.81,0.789,306307565,1.489429e+13,1.676,11.156327,0.220114
210,United States,2010,5675.786,18.368,31.449,0.102,2013.304,6.515,41.343,1308.832,...,0.372,650.38,2.105,247.17,0.800,309011469,1.527062e+13,1.688,11.732877,0.211351
211,United States,2011,5540.173,17.781,32.208,0.103,1903.555,6.109,45.542,1327.528,...,0.357,628.73,2.018,255.55,0.820,311584051,1.550930e+13,1.650,11.811093,0.232825
212,United States,2012,5338.698,17.000,35.270,0.112,1684.037,5.362,48.883,1388.255,...,0.337,618.71,1.970,246.42,0.785,314043885,1.584964e+13,1.567,12.761842,0.269340


In [37]:
# Drop the null rows
df = co2_temp_df.dropna()
df

,country,year,co2,co2_per_capita,cement_co2,cement_co2_per_capita,coal_co2,coal_co2_per_capita,flaring_co2,gas_co2,...,co2_per_gdp,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,energy_per_gdp,Avg Temperature,Avg Temperature Uncertainty
190,United States,1990,5113.455,20.282,33.484,0.133,1841.986,7.306,41.757,1033.673,...,0.553,767.47,3.044,250.21,0.992,252120309,9.250378e+12,2.432,11.990402,0.235575
191,United States,1991,5057.931,19.871,32.736,0.129,1827.639,7.180,41.137,1056.686,...,0.547,770.75,3.028,251.51,0.988,254539371,9.243499e+12,2.431,11.841255,0.194371
192,United States,1992,5167.481,20.108,32.993,0.128,1850.697,7.201,40.963,1091.238,...,0.540,769.59,2.995,257.86,1.003,256990608,9.572148e+12,2.383,11.091531,0.247250
193,United States,1993,5267.343,20.296,34.838,0.134,1908.247,7.353,40.820,1119.020,...,0.536,752.41,2.899,262.64,1.012,259532130,9.834941e+12,2.366,10.743381,0.215101
194,United States,1994,5358.824,20.435,36.310,0.138,1920.743,7.324,40.965,1145.894,...,0.524,759.44,2.896,270.67,1.032,262241204,1.023202e+13,2.325,11.319369,0.214190
195,United States,1995,5421.502,20.446,37.075,0.140,1942.302,7.325,39.410,1194.544,...,0.516,746.77,2.816,273.53,1.032,265163740,1.051026e+13,2.307,11.291533,0.196910
196,United States,1996,5610.582,20.909,37.309,0.139,2024.058,7.543,38.032,1215.071,...,0.514,740.53,2.760,281.25,1.048,268335007,1.090917e+13,2.297,10.781324,0.188526
197,United States,1997,5686.465,20.928,38.561,0.142,2067.227,7.608,38.183,1223.335,...,0.499,723.75,2.664,274.45,1.010,271713634,1.139876e+13,2.213,11.067458,0.214430
198,United States,1998,5731.050,20.827,39.461,0.143,2094.976,7.613,35.863,1200.762,...,0.481,704.80,2.561,271.12,0.985,275175309,1.190603e+13,2.134,12.423116,0.189361
199,United States,1999,5804.669,20.839,40.239,0.144,2100.317,7.540,35.834,1204.715,...,0.466,690.45,2.479,266.72,0.958,278548148,1.246382e+13,2.067,12.016275,0.195652


In [46]:
df.columns

Index(['country', 'year', 'co2', 'co2_per_capita', 'cement_co2',
       'cement_co2_per_capita', 'coal_co2', 'coal_co2_per_capita',
       'flaring_co2', 'gas_co2', 'gas_co2_per_capita', 'oil_co2',
       'oil_co2_per_capita', 'co2_growth_prct', 'co2_per_gdp', 'methane',
       'methane_per_capita', 'nitrous_oxide', 'nitrous_oxide_per_capita',
       'population', 'gdp', 'energy_per_gdp', 'Avg Temperature',
       'Avg Temperature Uncertainty'],
      dtype='object')

In [47]:
columns = [
        'country', 'year', 'co2_per_capita', 'cement_co2',
       'cement_co2_per_capita', 'coal_co2', 'coal_co2_per_capita',
       'flaring_co2', 'gas_co2', 'gas_co2_per_capita', 'oil_co2',
       'oil_co2_per_capita', 'co2_growth_prct', 'co2_per_gdp', 'methane',
       'methane_per_capita', 'nitrous_oxide', 'nitrous_oxide_per_capita',
       'population', 'gdp', 'energy_per_gdp', 'Avg Temperature',
       'Avg Temperature Uncertainty']

target = ['co2']

In [48]:
X = df.drop(columns = 'co2')
X = pd.get_dummies(X, columns = ['country'])
y = df['co2'].to_frame()

In [49]:
X.describe()

,year,co2_per_capita,cement_co2,cement_co2_per_capita,coal_co2,coal_co2_per_capita,flaring_co2,gas_co2,gas_co2_per_capita,oil_co2,...,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,energy_per_gdp,Avg Temperature,Avg Temperature Uncertainty,country_United States
count,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,...,24.00000,24.000000,24.000000,24.000000,2.400000e+01,2.400000e+01,24.000000,24.000000,24.000000,24.0
mean,2001.500000,19.946583,38.472667,0.135417,2015.415958,7.105917,39.903333,1217.710458,4.277917,2325.698000,...,688.96250,2.443500,259.413333,0.916625,2.847634e+08,1.294928e+13,1.997875,11.688566,0.218899,1.0
std,7.071068,1.273585,5.117198,0.017742,157.925290,0.658000,4.453662,91.849953,0.157986,171.557863,...,51.32097,0.358221,10.101674,0.087861,2.027296e+07,2.334810e+12,0.290745,0.580356,0.032669,0.0
min,1990.000000,17.000000,29.615000,0.097000,1684.037000,5.362000,35.834000,1033.673000,3.973000,2076.071000,...,618.71000,1.956000,241.810000,0.782000,2.521203e+08,9.243499e+12,1.567000,10.743381,0.188526,1.0
25%,1995.750000,19.775000,34.499500,0.128750,1905.346000,7.176000,36.761750,1191.637000,4.171750,2183.310000,...,652.19500,2.169000,251.185000,0.861500,2.675422e+08,1.080944e+13,1.742000,11.267063,0.199246,1.0
50%,2001.500000,20.415000,37.935000,0.139500,2045.642500,7.340000,38.723000,1213.093000,4.253500,2279.325000,...,666.57000,2.331000,258.900000,0.898500,2.859437e+08,1.321748e+13,1.969000,11.771985,0.212770,1.0
75%,2007.250000,20.803000,42.000750,0.147750,2162.515000,7.496000,41.008000,1257.562250,4.382750,2477.923750,...,742.09000,2.774000,264.755000,0.994750,3.013278e+08,1.513957e+13,2.299500,11.923896,0.229225,1.0
max,2013.000000,21.336000,46.851000,0.157000,2214.838000,7.738000,54.647000,1427.506000,4.528000,2608.477000,...,770.75000,3.044000,281.250000,1.048000,3.164005e+08,1.610843e+13,2.432000,13.098420,0.338102,1.0


In [50]:
y['co2'].value_counts()

5113.455    1
5057.931    1
5338.698    1
5540.173    1
5675.786    1
5478.210    1
5914.078    1
6131.004    1
6051.668    1
6134.521    1
6114.007    1
6011.837    1
5946.771    1
5904.882    1
6010.508    1
5804.669    1
5731.050    1
5686.465    1
5610.582    1
5421.502    1
5358.824    1
5267.343    1
5167.481    1
5474.257    1
Name: co2, dtype: int64

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [52]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
ranforest_model = BalancedRandomForestClassifier(n_estimators=100, random_state = 1)
ranforest_model

BalancedRandomForestClassifier(random_state=1)

In [53]:
# Calculated the balanced accuracy score
ranforest_model = ranforest_model.fit(X_train, y_train)
y_pred = ranforest_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

/Users/jiajaja/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


ValueError: Unknown label type: 'continuous'